In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-23"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10002,2024-10-23,Eua Nba,20:00,Detroit Pistons,Indiana Pacers,2.56,1.55,232.5,1.81,2.04,4.5,1.93,1.93,UPXi61NG,0.390625,0.645161,0.552486,0.490196,0.035786,0.4,0.8,NaN,NaN,183.646,49.962557,0.272059,196.444,47.053892,0.239528,169.658,293.072,140.65,268.32,0.0,0.0,0.0,0.0,0.347532,0.084485,0.000000,-1.10,-0.220,-7.090909,0.464660,0.4,-0.064660,-0.90,-0.180,-3.055556,0.483780,0.3,-0.183780
10003,2024-10-23,Eua Nba,20:30,Atlanta Hawks,Brooklyn Nets,1.34,3.47,220.5,1.81,2.04,-7.5,1.93,1.93,QiR04uhT,0.746269,0.288184,0.552486,0.490196,0.034453,0.6,0.6,NaN,NaN,223.214,70.835351,0.317343,216.562,51.346985,0.237101,192.784,198.276,165.54,150.10,0.0,0.0,0.0,0.0,0.626253,0.084485,0.000000,-2.91,-0.582,-0.584192,0.554385,0.5,-0.054385,-1.32,-0.264,-9.356061,0.490922,0.3,-0.190922
10004,2024-10-23,Eua Nba,20:30,Miami Heat,Orlando Magic,1.81,2.06,207.5,1.81,2.04,-2.5,2.00,1.84,MNCWbJFj,0.552486,0.485437,0.552486,0.490196,0.037923,0.4,0.8,NaN,NaN,225.136,57.959576,0.257443,189.600,27.595206,0.145544,270.248,217.250,326.40,168.78,0.0,0.0,0.0,0.0,0.091357,0.084485,0.058926,2.97,0.594,1.363636,0.483996,0.6,0.116004,-5.00,-1.000,-1.060000,0.491817,0.0,-0.491817
10005,2024-10-23,Eua Nba,20:30,Philadelphia 76ers,Milwaukee Bucks,2.36,1.63,223.5,1.84,2.00,2.5,2.04,1.81,06HBMM7c,0.423729,0.613497,0.543478,0.500000,0.037226,0.2,0.8,NaN,NaN,258.120,50.053555,0.193916,175.602,17.015496,0.096898,230.366,285.230,274.95,168.00,0.0,0.0,0.0,0.0,0.258741,0.058926,0.084485,0.04,0.008,170.000000,0.510455,0.2,-0.310455,-5.00,-1.000,-0.630000,0.452129,0.1,-0.352129
10006,2024-10-23,Eua Nba,20:30,Toronto Raptors,Cleveland Cavaliers,3.07,1.41,223.5,1.81,2.04,5.5,2.04,1.81,C8yVSGtp,0.325733,0.709220,0.552486,0.490196,0.034953,0.4,0.6,NaN,NaN,238.798,77.016425,0.322517,186.842,22.729322,0.121650,230.002,198.324,341.53,219.20,1.0,0.0,0.0,0.0,0.524017,0.084485,0.084485,1.24,0.248,8.346774,0.508110,0.5,-0.008110,-5.00,-1.000,-0.410000,0.511287,0.0,-0.511287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10108,2024-10-23,Rússia Copa Da Rússia,13:00,Tambov,Barnaul,1.87,13.00,144.5,1.83,1.83,-1.5,1.88,1.78,IBJIdwUQ,0.534759,0.076923,0.546448,0.546448,-0.388318,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,725.20,0.00,1.0,0.0,0.0,0.0,1.058520,0.000000,0.038640,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10109,2024-10-23,Rússia Copa Da Rússia,09:00,Novosibirsk F,Samara F,3.72,15.00,131.5,1.85,1.85,7.5,1.85,1.85,KYfCUgHl,0.268817,0.066667,0.540541,0.540541,-0.664516,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,195.96,0.0,0.0,0.0,0.0,0.852154,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10110,2024-10-23,Rússia Copa Da Rússia,10:00,Dynamo Moscow F,MBA Moscow F,6.40,20.00,133.5,1.85,1.85,11.5,1.95,1.72,K6YvW6hg,0.156250,0.050000,0.540541,0.540541,-0.793750,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,1147.00,0.0,1.0,0.0,0.0,0.728534,0.000000,0.088629,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10111,2024-10-23,Rússia Copa Da Rússia,13:00,Vologda F,Dynamo Kursk F,1.67,2.03,133.5,1.83,1.83,25.5,1.83,1.83,r7rZrry1,0.598802,0.492611,0.546448,0.546448,0.091413,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.0

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,20:00,Eua Nba,Detroit Pistons,Indiana Pacers,232.5,1.81,0.9835,Over
1,20:30,Eua Nba,Atlanta Hawks,Brooklyn Nets,220.5,1.81,0.9853,Over
2,20:30,Eua Nba,Miami Heat,Orlando Magic,207.5,1.81,0.8598,Over
3,20:30,Eua Nba,Philadelphia 76ers,Milwaukee Bucks,223.5,1.84,0.8436,Over
4,20:30,Eua Nba,Toronto Raptors,Cleveland Cavaliers,223.5,1.81,0.8454,Over
5,21:00,Eua Nba,Houston Rockets,Charlotte Hornets,228.5,1.81,0.8842,Over
6,21:00,Eua Nba,New Orleans Pelicans,Chicago Bulls,224.5,1.81,0.9848,Over
7,22:00,Eua Nba,Utah Jazz,Memphis Grizzlies,226.5,1.81,0.9708,Over
8,23:00,Eua Nba,Los Angeles Clippers,Phoenix Suns,223.5,1.84,0.9843,Over
9,23:00,Eua Nba,Portland Trail Blazers,Golden State Warriors,221.5,1.84,0.9717,Over
